In [ ]:
import eos
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import matplotlib.ticker as ticker
import wquantiles
import yaml

import scipy

from matplotlib.ticker import MultipleLocator
from matplotlib.legend_handler import HandlerTuple

from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from scipy.optimize import brentq

debug = False
resample = False

In [ ]:
plt.rc('text.latex', preamble=r'''
\usepackage{amsmath}
\usepackage{xcolor}
\usepackage{ulem}
''')

In [ ]:
def wstd(samples, weights=None):
    mean_w = np.average(samples, weights=weights)
    return np.sqrt(np.average((samples - mean_w)**2, weights=weights))

def get_gof(analysis, param_array):
    parameter_list = []

    for param in analysis.init_args['priors']:
        if param['type'] == 'transform':
            parameter_list += param['parameters']
        else:
            parameter_list.append(param['parameter'])

    for i, param in enumerate(parameter_list):
        analysis.parameters.set(param, param_array[i])

    return analysis.goodness_of_fit()

def match_table(l1, l2):
    L = []
    for el in l1:
        if el in l2:
            L.append(l2.index(el))
        else:
            L.append(None)
    return L

### Setup the storage space

In [ ]:
BASE_DIRECTORY = os.path.join(os.getcwd(), 'data')
if not os.path.exists(BASE_DIRECTORY):
    os.makedirs(os.path.abspath(BASE_DIRECTORY), exist_ok=True)

### Import analysis file

In [ ]:
af = eos.AnalysisFile(os.path.join(os.getcwd(), 'analysis.yaml'))

## Topological fit, no eta, CKM

In [ ]:
posterior = 'Topological_no_eta_CKM'
analysis = af.analysis(posterior)

In [ ]:
start_point = [
        0.71886282,  0.41355453, -0.24192837,  0.05145696, -0.13377584,
       -0.06808363,  0.05602922,  0.1371432 ,  0.09804935, -0.08362717,
        0.1024549 ,  0.22550703, -0.13527755, -0.22046345,  0.13750201,
       -0.05900109, -0.06218254,  0.03639814,  0.01915742
] + [0.81974778, 0.22498966,  0.16070297,  0.35581278]

In [ ]:
bfp, gof = eos.tasks.find_mode(af, posterior, start_point = start_point, base_directory=BASE_DIRECTORY)
#bfp, gof = eos.tasks.find_mode(af, posterior, importance_samples=True, base_directory=BASE_DIRECTORY, optimizations=100)
display(gof)
bfp.point

In [ ]:
if resample:
    eos.tasks.sample_nested(af, posterior, base_directory=BASE_DIRECTORY, bound='multi', nlive=750, dlogz=0.01)

f = eos.data.ImportanceSamples(os.path.join(BASE_DIRECTORY, posterior, 'samples'))
print(f"Collected {f.samples.shape[0]} samples")

In [ ]:
for i, p in enumerate(analysis.varied_parameters):
    mean = np.average(f.samples[:,i], weights=f.weights)
    std = wstd(f.samples[:,i], weights=f.weights)
    print(f"      - {{ 'parameter': '{p.name()}', 'min':  {mean - 6*std:.5f}, 'max':  {mean + 6*std:.5f}, 'type': 'uniform'}}")

In [ ]:
if resample:
    eos.tasks.corner_plot(af, posterior, base_directory=BASE_DIRECTORY)

In [ ]:
if resample:
    eos.tasks.predict_observables(af, posterior, "BRs", base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "BRratios",   base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ACPs",      base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "SCPs",      base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ADGs",      base_directory=BASE_DIRECTORY)

    eos.tasks.predict_observables(af, posterior, "BRs_eta",   base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ACPs_eta",  base_directory=BASE_DIRECTORY)

In [ ]:
BRs_light = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-BRs")

BRs_labels = ["$"+eos.Observables()[obs].latex()+"$" for obs in BRs_light.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['BRs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(BRs_light.lookup_table.keys()), [c['observable'] for c in constraints])

BRs_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
BRs_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
BRs_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

used_meas_mean = np.copy(BRs_meas_mean)
used_meas_lo = np.copy(BRs_meas_lo)
used_meas_hi = np.copy(BRs_meas_hi)

BRs_meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")] = 0.766e-6
BRs_meas_lo[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")]   = 0.096e-6
BRs_meas_hi[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")]   = 0.096e-6

BRs_meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")] = 6.19e-6
BRs_meas_lo[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")]   = 0.74e-6
BRs_meas_hi[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")]   = 0.74e-6

BRs_meas_mean[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")] = 0.079e-6
BRs_meas_lo[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")]   = 0.015e-6
BRs_meas_hi[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")]   = 0.015e-6

pred_mean = np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5 + 0.34)) - pred_mean

BRs_meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->Kbar^0pi^0::BR_exp")] = pred_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->Kbar^0pi^0::BR_exp")]
mask = BRs_meas_hi > 0

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(BRs_meas_mean)))
ax.errorbar(BRs_meas_mean[mask]/BRs_meas_mean[mask], np.arange(len(BRs_meas_mean))[mask] + 0.1, xerr=[BRs_meas_lo[mask]/BRs_meas_mean[mask], BRs_meas_hi[mask]/BRs_meas_mean[mask]],
            color="grey",  fmt="d")
ax.errorbar(used_meas_mean/BRs_meas_mean, np.arange(len(BRs_meas_mean)) + 0.1, xerr=[used_meas_lo/BRs_meas_mean, used_meas_hi/BRs_meas_mean],
            color="k",  fmt="d", label=r"Measurement")
ax.errorbar(pred_mean/BRs_meas_mean, np.arange(len(BRs_meas_mean)), xerr=[pred_lo/BRs_meas_mean, pred_hi/BRs_meas_mean],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")

ax.yaxis.set_ticks(range(len(BRs_meas_mean)))
ax.yaxis.set_ticklabels(BRs_labels)

ax.set_xlabel(r"$\mathcal{B}/\mathcal{B}_{\mathrm{measured}}$")
ax.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.0 + 1. / len(pred_mean)))

ax.hlines(7.5, 0, 3, lw=1, color="k")
ax.set_xlim(0., 2.2)
ax.set_ylim(-0.5, len(pred_mean) - 0.5)

for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRs_labels[i]}', 'mean': '{p}', 'std low': '{pred_lo[i]}', 'std high': '{pred_hi[i]}'}}")

#fig.savefig("./figures/BRs.pdf")
fig.show()

In [ ]:
ACPs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ACPs")

ACP_labels = ["$"+eos.Observables()[obs].latex()+"$" for obs in ACPs.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['ACPs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(ACPs.lookup_table.keys()), [c['observable'] for c in constraints])

ACP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

ACP_meas_mean[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")] = 0.172
ACP_meas_lo[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")]   = 0.031
ACP_meas_hi[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")]   = 0.031

ACP_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5))
ACP_pred_lo   = ACP_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5 - 0.34))
ACP_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5 + 0.34)) - ACP_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ACP_meas_mean)))
ax.errorbar(ACP_pred_mean, np.arange(len(ACP_meas_mean)), xerr=[ACP_pred_lo, ACP_pred_hi],  color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")
ax.errorbar(ACP_meas_mean, np.arange(len(ACP_meas_mean)) + 0.1, xerr=[ACP_meas_lo, ACP_meas_hi],  color="k",  fmt="d", label=r"Measurement")

ax.yaxis.set_ticks(range(len(ACP_meas_mean)))
ax.yaxis.set_ticklabels(ACP_labels)

ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}$")
ax.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.0 + 1. / len(ACP_meas_mean)))

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(7.5, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="k")

for i, p in enumerate(ACP_pred_mean):
    print(f"      - {{ 'decay': '{ACP_labels[i]}', 'mean': '{p*100}', 'std low': '{ACP_pred_lo[i]*100}', 'std high': '{ACP_pred_hi[i]*100}'}}")

# fig.savefig("./figures/ACPs.pdf")
fig.show()

In [ ]:
SCPs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-SCPs")

SCP_labels = ["$"+eos.Observables()[obs].latex()+"$" for obs in SCPs.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['SCPs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(SCPs.lookup_table.keys()), [c['observable'] for c in constraints])

SCP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
SCP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
SCP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

SCP_meas_mean[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")] = -0.139
SCP_meas_lo[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.032
SCP_meas_hi[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.032

SCP_pred_mean = np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5))
SCP_pred_lo   = SCP_pred_mean - np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5 - 0.34))
SCP_pred_hi   = np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5 + 0.34)) - SCP_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(SCP_labels)))
ax.errorbar(SCP_pred_mean, np.arange(len(SCP_labels)),       xerr=[SCP_pred_lo, SCP_pred_hi],  color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")
ax.errorbar(SCP_meas_mean, np.arange(len(SCP_labels)) + 0.1, xerr=[SCP_meas_lo, SCP_meas_hi],  color="k",  fmt="d", label=r"Measurement")

ax.yaxis.set_ticks(range(len(SCP_labels)))
ax.yaxis.set_ticklabels(SCP_labels)

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(4.5, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="k")

ax.set_xlabel(r"$\mathcal{S}_\mathrm{CP}$")
ax.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.0 + 1. / len(SCP_labels)))

for i, p in enumerate(SCP_pred_mean):
    print(f"      - {{ 'decay': '{SCP_labels[i]}', 'median': '{p*100}', 'std low': '{SCP_pred_lo[i]*100}', 'std high': '{SCP_pred_hi[i]*100}'}}")

# fig.savefig("./figures/SCPs.pdf")
fig.show()

In [ ]:
ADGs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ADGs")

ADG_labels = ["$"+eos.Observables()[obs].latex().replace("A^{\\Delta\\Gamma}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in ADGs.lookup_table.keys()]

# exp_const   = [const.constraint for const in af.likelihoods['ADGs'].constraints]
# constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

# tab = match_table(list(ADGs.lookup_table.keys()), [c['observable'] for c in constraints])

# ADG_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
# ADG_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
# ADG_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

# ADG_meas_mean[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")] = -0.083
# ADG_meas_lo[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.010
# ADG_meas_hi[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.010

ADG_pred_mean = np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5))
ADG_pred_lo   = ADG_pred_mean - np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5 - 0.34))
ADG_pred_hi   = np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5 + 0.34)) - ADG_pred_mean

# fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ADG_labels)))
# ax.errorbar(ADG_meas_mean, np.arange(len(ADG_meas_mean)) - 0.2, xerr=[ADG_meas_lo, ADG_meas_hi],
#             color="k",  fmt="d", label=r"Measurement")
# ax.errorbar(ADG_pred_mean, np.arange(len(ADG_meas_mean)) + 0.2, xerr=[ADG_pred_lo, ADG_pred_hi],
#             color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

# ax.yaxis.set_ticks(range(len(ADG_labels)))
# ax.yaxis.set_ticklabels(ADG_labels)

# ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{mix}$")
# # ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(ADG_labels)))

# ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
# ax.hlines(4.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
# ax.set_ylim(-0.5, len(ADG_pred_mean) - 0.3)

# ax.text(0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
#                     transform=ax.transAxes,
#                     color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
#                     horizontalalignment="left", verticalalignment="top", zorder=+5)

for i, p in enumerate(ADG_pred_mean):
    print(f"      - {{ 'decay': '{ADG_labels[i]}', 'median': '{p*100:.2f}', 'std low': '{ADG_pred_lo[i]*100:.2f}', 'std high': '{ADG_pred_hi[i]*100:.2f}'}}")

# fig.savefig("./figures/ADGs.pdf")
# fig.show()

In [ ]:
BRratios = eos.Prediction(BASE_DIRECTORY+posterior+"/pred-BRratios")

BRratio_labels = ["$"+eos.Observables()[obs].latex()+"$" for obs in BRratios.lookup_table.keys()]

exp_const   = [const.name for const in af._likelihoods['BRratios'].manual_constraints]
constraints = [const.info for const in af._likelihoods['BRratios'].manual_constraints]

BRratios_mean = np.array([float(c["mean"]) for c in constraints])
BRratios_lo   = np.array([np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints])
BRratios_hi   = np.array([np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints])

pred_mean = np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5 + 0.34)) - pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(BRratio_labels)))
ax.errorbar(BRratios_mean/BRratios_mean, np.arange(len(BRratio_labels))+0.1, xerr=[BRratios_lo/BRratios_mean, BRratios_hi/BRratios_mean],  color="k",  fmt="d", label=r"Measurement")
ax.errorbar(pred_mean/BRratios_mean, np.arange(len(BRratio_labels)), xerr=[pred_lo/BRratios_mean, pred_hi/BRratios_mean],      color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")

ax.yaxis.set_ticks(range(len(BRratio_labels)))
ax.yaxis.set_ticklabels(BRratio_labels)

ax.set_xlabel(r"$\mathcal{B}/\mathcal{B}_{\mathrm{measured}}$")
ax.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1. + 1. / len(BRratio_labels)))


for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRratio_labels[i]}', 'mean': '{p}', 'std low': '{pred_lo[i]}', 'std high': '{pred_hi[i]}'}}")

# fig.savefig("./figures/BRratios.pdf")
fig.show()

In [ ]:
BRs_eta = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-BRs_eta")

BRs_eta_labels = ["$"+eos.Observables()[obs].latex().replace("\\mathcal{B}_\\mathrm{exp}(", "").replace(")", "")+"$"for obs in BRs_eta.lookup_table.keys()]

exp_const   = ['B^+->etapi^+::BR_exp@PDG:2024A',
 'B^0->etapi^0::BR_exp@Belle:2015B',
 'B^+->etaK^+::BR_exp@PDG:2024A',
 'B^0->etaeta::BR_exp@BaBar:2009B',
 'B_s^0->etaeta::BR_exp@Belle:2021A',
 'B^0->etaK^0::BR_exp@PDG:2024A'
 ]

constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(BRs_eta.lookup_table.keys()), [c['observable'] for c in constraints])

meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

pred_mean = np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5 + 0.34)) - pred_mean

meas_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etaK^0::BR_exp")] = pred_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etaK^0::BR_exp")]
meas_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etapi^0::BR_exp")] = pred_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etapi^0::BR_exp")]
mask = meas_hi > 0

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(meas_mean)))
ax.errorbar(meas_mean[mask]/pred_mean[mask], np.arange(len(meas_mean))[mask] + 0.1, xerr=[meas_lo[mask]/pred_mean[mask], meas_hi[mask]/pred_mean[mask]],
            color="grey",  fmt="d", label="Measurement")
ax.errorbar(pred_mean/pred_mean, np.arange(len(meas_mean)), xerr=[pred_lo/pred_mean, pred_hi/pred_mean],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")

ax.yaxis.set_ticks(range(len(meas_mean)))
ax.yaxis.set_ticklabels(BRs_eta_labels)

ax.set_xlabel(r"$\mathcal{B}/\mathcal{B}_{\mathrm{measured}}$")
ax.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.0 + 1. / len(pred_mean)))

ax.set_xlim(-0.5, 10)
ax.hlines(3.5, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(pred_mean) - 0.5)

for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRs_eta_labels[i]}', 'mean': '{10**6*p:.3f}', 'std low': '{10**6*pred_lo[i]:.3f}', 'std high': '{10**6*pred_hi[i]:.3f}'}}")

fig.savefig("./figures/BRs_eta.pdf")
fig.show()

In [ ]:
ACPs_eta = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ACPs_eta")

ACP_eta_labels = ["$"+eos.Observables()[obs].latex().replace("A^\\mathrm{dir}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in ACPs_eta.lookup_table.keys()]

exp_const   = ['B^+->etaK^+::A_CP@PDG:2024A',
 'B^+->etapi^+::A_CP@PDG:2024A'
]

constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(ACPs_eta.lookup_table.keys()), [c['observable'] for c in constraints])

ACP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

ACP_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5))
ACP_pred_lo   = ACP_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5 - 0.34))
ACP_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5 + 0.34)) - ACP_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ACP_meas_mean)))
ax.errorbar(ACP_meas_mean, np.arange(len(ACP_meas_mean)) + 0.1, xerr=[ACP_meas_lo, ACP_meas_hi],
            color="grey",  fmt="d", label=r"Measurement")
ax.errorbar(ACP_pred_mean, np.arange(len(ACP_meas_mean)), xerr=[ACP_pred_lo, ACP_pred_hi],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_F$")

ax.yaxis.set_ticks(range(len(ACP_meas_mean)))
ax.yaxis.set_ticklabels(ACP_eta_labels)

ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}$")
# ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(ACP_meas_mean)))

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(3.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(ACP_pred_mean) - 0.3)

ax.text(1 - 0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

for i, p in enumerate(ACP_pred_mean):
    print(f"      - {{ 'decay': '{ACP_eta_labels[i]}', 'mean': '{p*100:.3f}', 'std low': '{ACP_pred_lo[i]*100:.3f}', 'std high': '{ACP_pred_hi[i]*100:.3f}'}}")

fig.savefig("./figures/ACPs_eta.pdf")
fig.show()

In [ ]:
ckmlam    = analysis.parameters["CKM::lambda"].evaluate()
ckmetabar = analysis.parameters["CKM::etabar"].evaluate()
ckmrhobar = analysis.parameters["CKM::rhobar"].evaluate()

ckmratio = ckmlam**2 * (4 * ckmetabar**2 + (-2 + ckmlam**2 + 2 * ckmrhobar)**2) / (-2 + ckmlam**2)**2
ckmratio

In [ ]:
fig, ax = plt.subplots()

plt.xlabel(BRs_labels[6])
plt.ylabel(BRs_labels[14])

xmedian  = wquantiles.quantile(BRs_light.samples[:,6], BRs_light.weights, 0.5)
xstdminus = wquantiles.quantile(BRs_light.samples[:,6], BRs_light.weights, 0.66) - xmedian
xstdplus  = xmedian - wquantiles.quantile(BRs_light.samples[:,6], BRs_light.weights, 0.34)
xmin = xmedian - 15 * xstdminus
xmax = xmedian + 15 * xstdplus

ymedian  = wquantiles.quantile(BRs_light.samples[:,14], BRs_light.weights, 0.5)
ystdminus = wquantiles.quantile(BRs_light.samples[:,14], BRs_light.weights, 0.66) - ymedian
ystdplus  = ymedian - wquantiles.quantile(BRs_light.samples[:,14], BRs_light.weights, 0.34)
ymin = ymedian - 15 * ystdminus
ymax = ymedian + 15 * ystdplus

kde = gaussian_kde(BRs_light.samples[:,(6, 14)].T, weights=BRs_light.weights)
kde.set_bandwidth(bw_method='silverman')
kde.set_bandwidth(bw_method=kde.factor * 1.4)

xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
pdf = np.reshape(kde(positions).T, xx.shape)
pdf /= pdf.sum()

# find the PDF value corresponding to a given cummulative probability
plevel = lambda x, pdf, P: pdf[pdf > x].sum() - P
plevels = []
labels = []
for level in [0, 68.3, 95.4, 99.7]:
    plevels.append(brentq(plevel, 0., 1., args=(pdf, level / 100.0)))
    labels.append(f'{level}%')

CS = ax.contour(pdf.transpose(),
            colors="k",
            extent=[xmin, xmax, ymin, ymax],
            levels=plevels[::-1])

fmt = {}
strs = [r'$3 \sigma$', r'$2 \sigma$', r'$1 \sigma$']
for l, s in zip(CS.levels, strs):
    fmt[l] = s

# Label every other level using strings
plt.clabel(CS, inline=True, fmt=fmt, fontsize=10)

ax.text(0.04, 1 - 0.04, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="left", verticalalignment="top", zorder=+5)

ax.fill_between(np.linspace(xmin, xmax, 10), ckmratio * np.linspace(xmin, xmax, 10), color = "Grey", alpha = 0.5, label=r"Excluded by $SU(3)_\mathrm{F}$")
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.add_patch(matplotlib.patches.Rectangle((BRs_meas_mean[6] - BRs_meas_lo[6], ymin), BRs_meas_hi[6] + BRs_meas_lo[6], ymax - ymin, alpha = 0.4, color = "C1", label = r"PDG 2024"))
ax.add_patch(matplotlib.patches.Rectangle((xmin, BRs_meas_mean[14] - BRs_meas_lo[14]), xmax - xmin, BRs_meas_hi[14] + BRs_meas_lo[14], alpha = 0.4, color = "C1"))

handles, labels = plt.gca().get_legend_handles_labels()

proxy = [plt.Line2D((0, 1), (0.5, 0.), color="k")]

plt.legend(proxy + handles,
           [r"$\mathrm{SU}(3)_F$ fit"] + labels,
            loc='center right', ncol=1, prop={'size': 13}, bbox_to_anchor=(1.5, 0.5), labelspacing=1)

plt.savefig("./figures/BRrelations.pdf")
plt.show()

#### dtheta plot

In [ ]:
def evaluate_obs(sample):
    for p, v in zip(analysis.varied_parameters, sample):
        p.set(v)
    options = eos.Options({"q": "d", "P1": "pi^-", "P2": "pi^+", "representation": "topological", "model": "CKM"})
    return [
        eos.Observable.make("B->PP::r_q",     analysis.parameters, eos.Kinematics(), options).evaluate(),
        eos.Observable.make("B->PP::theta_q", analysis.parameters, eos.Kinematics(), options).evaluate()
    ]

from tqdm.contrib.concurrent import process_map
dtheta = np.array(process_map(evaluate_obs, f.samples, max_workers=30, chunksize=5))

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(dtheta[:,1], dtheta[:,0], weights=f.weights, bins=[np.linspace(2.4, 2.7, 100), np.linspace(0.2, 0.35, 100)], cmap="Greys")
ax.set_xlabel(r'$\theta$ [rad]')
ax.set_ylabel(r"$r$")
ax.set_title(r"$B^0\to\pi^+\pi^-$")

def deg2rad(x): return x * np.pi / 180
def rad2deg(x): return x * 180 / np.pi

secax = ax.secondary_xaxis('top', functions=(rad2deg, deg2rad))
secax.set_xlabel(r'$\theta$ [degrees]')
plt.show()

In [ ]:
def acppipi(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    return - 2 * d * np.sin(theta) * np.sin(gamma) \
        / (1 - 2 * d * np.cos(theta) * np.cos(gamma) + d**2)

def scppipi(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    phid = 0.79726
    return (d**2 * np.sin(phid) - 2 * d * np.cos(theta) * np.sin(phid + gamma) + np.sin(phid + 2 * gamma)) \
        / (1 - 2 * d * np.cos(theta) * np.cos(gamma) + d**2 )

def acpKK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    eps = 0.053319
    return 2 * eps * d * np.sin(theta) * np.sin(gamma) \
        / (eps**2 + 2 * eps * d * np.cos(theta) * np.cos(gamma) + d**2)

def scpKK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    phis = -0.03764
    eps = 0.053319
    return (d**2 * np.sin(phis) + 2 * eps * d * np.cos(theta) * np.sin(phis + gamma) + eps**2 * np.sin(phis + 2 * gamma)) \
        / (eps**2 + 2 * eps * d * np.cos(theta) * np.cos(gamma) + d**2 )

def acpdgKK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    phis = -0.03764
    eps = 0.053319
    return - (d**2 * np.cos(phis) + 2 * eps * d * np.cos(theta) * np.cos(phis + gamma) + eps**2 * np.cos(phis + 2 * gamma)) \
        / (eps**2 + 2 * eps * d * np.cos(theta) * np.cos(gamma) + d**2 )


thetabins = np.linspace(2.0, 2.96, 150)
rbins = np.linspace(0.033, 0.4, 150)

theta_mesh, d_mesh = np.meshgrid(thetabins, rbins)
acppipi_data = acppipi(d_mesh, theta_mesh)
scppipi_data = scppipi(d_mesh, theta_mesh)
acpKK_data   = acpKK(d_mesh, theta_mesh)
scpKK_data   = scpKK(d_mesh, theta_mesh)
acpdgKK_data = acpdgKK(d_mesh, theta_mesh)

fig, ax = plt.subplots()

# ax.hist2d(dtheta[:,1], dtheta[:,0], weights=f.weights, bins=[thetabins, rbins], cmap="Greys")

acpdgKK_contour = ax.contourf(thetabins, rbins, acpdgKK_data, levels = [-0.897 - 0.087, -0.897 + 0.087], alpha=0.4, cmap="Purples")
scpKK_contour   = ax.contourf(thetabins, rbins, scpKK_data,   levels = [-0.139 - 0.032, -0.139 + 0.032], alpha=0.4, cmap="Blues")
acpKK_contour   = ax.contourf(thetabins, rbins, acpKK_data,   levels = [ 0.172 - 0.031,  0.172 + 0.031], alpha=0.4, cmap="twilight")
acppipi_contour = ax.contourf(thetabins, rbins, acppipi_data, levels = [-0.314 - 0.03,  -0.314 + 0.03],  alpha=0.4, cmap="YlOrRd")
scppipi_contour = ax.contourf(thetabins, rbins, scppipi_data, levels = [ 0.670 - 0.03,   0.670 + 0.03],  alpha=0.4, cmap="Reds")

kde = gaussian_kde(dtheta[:,(1, 0)].T, weights=f.weights)
kde.set_bandwidth(bw_method='silverman')
kde.set_bandwidth(bw_method=kde.factor * 3)

xx, yy = np.mgrid[thetabins[0]:thetabins[-1]:100j, rbins[0]:rbins[-1]:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
pdf = np.reshape(kde(positions).T, xx.shape)
pdf /= pdf.sum()

# find the PDF value corresponding to a given cummulative probability
plevel = lambda x, pdf, P: pdf[pdf > x].sum() - P
plevels = []
labels = []
for level in [0, 68.3, 95.4, 99.7]:
    plevels.append(brentq(plevel, 0., 1., args=(pdf, level / 100.0)))
    labels.append(f'{level}%')

CS = ax.contour(pdf.transpose(),
            colors="k",
            extent=[thetabins[0], thetabins[-1], rbins[0], rbins[-1]],
            levels=plevels[::-1])


fmt = {}
strs = [r'$3 \sigma$', r'$2 \sigma$', r'$1 \sigma$']
for l, s in zip(CS.levels, strs):
    fmt[l] = s

# Label every other level using strings
plt.clabel(CS, inline=True, fmt=fmt, fontsize=10)

ax.set_xlabel(r'$\theta_p$ [rad]')
ax.set_ylabel(r"$r_p$")
# ax.set_title(r"$B^0\to\pi^+\pi^-$ and $B_s^0\to K^+ K^-$")

ax.errorbar(2.58371, 0.212435, xerr = [[0.08], [0.08]], yerr = [[0.02], [0.03]], color="grey", marker="*", markersize=10)#, label = r"$B^0\to\pi^+\pi^-$ best fit")
ax.errorbar(2.14402, 0.190706, xerr = [[0.24], [0.23]], yerr = [[0.04], [0.06]], color="grey", marker="x", markersize=10)#, label = r"$B_s^0\to K^+K^-$ best fit")

def deg2rad(x): return x * np.pi / 180
def rad2deg(x): return x * 180 / np.pi

secax = ax.secondary_xaxis('top', functions=(rad2deg, deg2rad))
secax.set_xlabel(r'$\theta_p$ [degrees]', labelpad=10)

ax.text(0.04, 0.04, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="left", verticalalignment="bottom", zorder=+5)

handles, labels = plt.gca().get_legend_handles_labels()

proxy = [plt.Rectangle((0, 0), 1, 1, fc = pc.get_facecolor()[0])
         for pc in acpKK_contour.collections + scpKK_contour.collections + acpdgKK_contour.collections + acppipi_contour.collections + scppipi_contour.collections] \
      + [plt.Line2D((0, 1), (0.5, 0.), color="k")]

ax.set_xbound(2.0, 2.9)
ax.set_ybound(0.015, 0.4)

plt.legend(proxy + handles,
            [r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}(B_s^0\to K^+K^-)$", r"$\mathcal{A}_\mathrm{CP}^\mathrm{mix}(B_s^0\to K^+K^-)$",
             r"$\mathcal{A}_\mathrm{CP}^{\Delta\Gamma}(B_s^0\to K^+K^-)$", r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}(B^0\to\pi^+\pi^-)$",
             r"$\mathcal{A}_\mathrm{CP}^\mathrm{mix}(B^0\to\pi^+\pi^-)$", r"$\mathrm{SU}(3)_F$ fit"] + labels,
           loc='center right', ncol=1, prop={'size': 13}, bbox_to_anchor=(1.5, 0.5), labelspacing=1)

plt.savefig("./figures/dtheta.pdf")
plt.show()

## QCDF CKM

In [ ]:
posterior = 'QCDF_no_eta_CKM'
analysis = af.analysis(posterior)

In [ ]:
start_point = [
        5.90929561e-01,  4.36141077e-01, -2.70295148e-01,  2.52392255e+02,
       -1.22928286e+02, -6.65808575e+00,  1.97880148e+01,  4.68987923e+00,
       -2.24506350e+00,  1.50188420e+01, -3.52993176e+00, -3.23588064e-01,
        1.37187560e-01,  3.85879336e-02, -1.16569831e-02, -1.00687059e+01,
        6.46589752e+00,  3.72152239e+01, -1.49881622e+01, -1.02459648e-01,
        4.32456914e-02, -5.36475906e+01,  1.07144129e+01,  8.19750275e-01,
        2.24990084e-01,  1.61013002e-01,  3.53952428e-01
]

In [ ]:
bfp, gof = eos.tasks.find_mode(af, posterior, start_point = start_point, optimizations=100, base_directory=BASE_DIRECTORY)
#bfp, gof = eos.tasks.find_mode(af, posterior, importance_samples=True, optimizations=25, base_directory=BASE_DIRECTORY)
display(gof)
display(bfp.point)

In [ ]:
if resample:
    eos.tasks.sample_nested(af, posterior, base_directory=BASE_DIRECTORY, bound='multi', nlive=750, dlogz=0.01)

f = eos.data.ImportanceSamples(os.path.join(BASE_DIRECTORY, posterior, 'samples'))
print(f"Collected {f.samples.shape[0]} samples")

In [ ]:
if debug:
    eos.tasks.corner_plot(af, posterior, base_directory=BASE_DIRECTORY)

In [ ]:
for i, p in enumerate(analysis.varied_parameters):
    mean = np.average(f.samples[:,i], weights=f.weights)
    std = wstd(f.samples[:,i], weights=f.weights)
    print(f"      - {{ 'parameter': '{p.name()}', 'min':  {mean - 5*std:.5f}, 'max':  {mean + 5*std:.5f}, 'type': 'uniform'}}")

In [ ]:
samples_mean = np.array(wquantiles.quantile(np.transpose(f.samples), f.weights, 0.5))
samples_lo   = samples_mean - np.array(wquantiles.quantile(np.transpose(f.samples), f.weights, 0.5 - 0.34))
samples_hi   = np.array(wquantiles.quantile(np.transpose(f.samples), f.weights, 0.5 + 0.34)) - samples_mean

for i, p in enumerate(analysis.varied_parameters[:-4]):
    print(f"{p.latex()}' = {samples_mean[i]:.3f}^{{+ {samples_hi[i]:.3f}}}_{{- {samples_lo[i]:.3f}}}")

for i, p in enumerate(analysis.varied_parameters[-4:]):
    print(f"{p.latex()}' = {samples_mean[i-4]:.5f}^{{+ {samples_hi[i-4]:.5f}}}_{{- {samples_lo[i-4]:.5f}}}")


In [ ]:
mean = wquantiles.quantile(np.sqrt(f.samples[::,1]**2 + f.samples[::,2]**2), f.weights, 0.5)
lo   = mean - wquantiles.quantile(np.sqrt(f.samples[::,1]**2 + f.samples[::,2]**2), f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(np.sqrt(f.samples[::,1]**2 + f.samples[::,2]**2), f.weights, 0.5 + 0.34) - mean

print(f"|a2 + a4u| = {mean:.3f} - {lo:.3f} + {hi:.3f}")

mean = wquantiles.quantile(1.5 * f.samples[::,13] - f.samples[::,19], f.weights, 0.5)
lo   = mean - wquantiles.quantile(1.5 * f.samples[::,13] - f.samples[::,19], f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(1.5 * f.samples[::,13] - f.samples[::,19], f.weights, 0.5 + 0.34) - mean

print(f"Re 3/2 * a3EWc - a4c = {mean:.3f} - {lo:.3f} + {hi:.3f}")

mean = wquantiles.quantile(1.5 * f.samples[::,14] - f.samples[::,20], f.weights, 0.5)
lo   = mean - wquantiles.quantile(1.5 * f.samples[::,14] - f.samples[::,20], f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(1.5 * f.samples[::,14] - f.samples[::,20], f.weights, 0.5 + 0.34) - mean

print(f"Im 3/2 * a3EWc - a4c = {mean:.3f} - {lo:.3f} + {hi:.3f}")

In [ ]:
mean = wquantiles.quantile(1.5 * f.samples[::,15] - 0.25 * f.samples[::,21], f.weights, 0.5)
lo   = mean - wquantiles.quantile(1.5 * f.samples[::,15] - 0.25 * f.samples[::,21], f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(1.5 * f.samples[::,15] - 0.25 * f.samples[::,21], f.weights, 0.5 + 0.34) - mean

print(f"Re 3/2 * b4EWc - b4c = {mean:.3f} - {lo:.3f} + {hi:.3f}")

mean = wquantiles.quantile(1.5 * f.samples[::,16] - 0.25 * f.samples[::,20], f.weights, 0.5)
lo   = mean - wquantiles.quantile(1.5 * f.samples[::,16] - 0.25 * f.samples[::,20], f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(1.5 * f.samples[::,16] - 0.25 * f.samples[::,20], f.weights, 0.5 + 0.34) - mean

print(f"Im 3/2 * b4EWc - b4c = {mean:.3f} - {lo:.3f} + {hi:.3f}")

In [ ]:
samples = np.copy(f.samples)[::,(0,1,2,7,8,19,20,11,12,13,14)]

samples[:,9]  = 1.5 * samples[:,9]  - samples[:,5]
samples[:,10] = 1.5 * samples[:,10] - samples[:,6]
samples[:,5] = -samples[:,5]
samples[:,6] = -samples[:,6]
samples[:,7] = samples[:,5] - 1.5 * samples[:,7]
samples[:,8] = samples[:,6] - 1.5 * samples[:,8]


weights = f.weights
labels = [
 '$\\mathrm{Re}\\,(\\alpha_1 + \\alpha_4^u)$',
 '$\\mathrm{Re}\\,(\\alpha_2 - \\alpha_4^u)$',
 '$\\mathrm{Im}\\,(\\alpha_2 - \\alpha_4^u)$',
 '$\\mathrm{Re}\\,\\alpha_4^u$',
 '$\\mathrm{Im}\\,\\alpha_4^u$',
 '$\\mathrm{Re}\\,(\\frac{3}{2} \\alpha_{4,EW}^c + \\alpha_4^c)$',
 '$\\mathrm{Im}\\,(\\frac{3}{2} \\alpha_{4,EW}^c + \\alpha_4^c)$',
 '$\\mathrm{Re}\\,\\alpha_4^c$',
 '$\\mathrm{Im}\\,\\alpha_4^c$',
 '$\\mathrm{Re}\\,(\\frac{3}{2} \\alpha_{3,EW}^c - \\alpha_4^c)$',
 '$\\mathrm{Im}\\,(\\frac{3}{2} \\alpha_{3,EW}^c - \\alpha_4^c)$',
#  '$\\mathrm{Re}\\,\\alpha_{3,EW}^c + \\alpha_{4,EW}^c$',
#  '$\\mathrm{Im}\\,\\alpha_{3,EW}^c + \\alpha_{4,EW}^c$'
]



size = samples.shape[-1]

fig, axes = plt.subplots(size, size, figsize=(3.0 * size, 3.0 * size), dpi=100)

for i in range(size):
    # diagonal
    ax = axes[i, i]
    variable_samples = samples[:, i] # For a single variable

    ax.hist(variable_samples, weights=weights, alpha=0.5, bins=100, density=True, stacked=True, color='C1')
    xmin = np.min(variable_samples)
    xmax = np.max(variable_samples)
    ax.set_xlim((xmin, xmax))
    ax.set_xlabel(labels[i])
    ax.set_ylabel(labels[i])
    ax.set_aspect(np.diff((xmin, xmax))[0] / np.diff(ax.get_ylim())[0])

    for j in range(0, size):
        # off-diagonal
        if j < i:
            axes[i, j].set_axis_off()

        if j <= i:
            continue

        ax = axes[i, j]

        # Samples for two single variables as x and y data
        xsamples = samples[:, j]
        ysamples = samples[:, i]

        xmin = np.min(xsamples)
        xmax = np.max(xsamples)
        ymin = np.min(ysamples)
        ymax = np.max(ysamples)
        ax.hist2d(xsamples, ysamples, weights=weights, alpha=1.0, bins=100, cmap='Greys', rasterized=True)
        ax.set_xlim((xmin, xmax))
        ax.set_ylim((ymin, ymax))
        ax.set_aspect(np.diff((xmin, xmax))[0] / np.diff((ymin, ymax))[0])

fig.tight_layout()
fig.show()

In [ ]:
samples = np.copy(f.samples)[::,(0,1,2,7,8,19,20,11,12,13,14)]

samples[:,9]  = 1.5 * samples[:,9]  - samples[:,5]
samples[:,10] = 1.5 * samples[:,10] - samples[:,6]
samples[:,5] = -samples[:,5]
samples[:,6] = -samples[:,6]
samples[:,7] = samples[:,5] - 1.5 * samples[:,7]
samples[:,8] = samples[:,6] - 1.5 * samples[:,8]

# bfp
bfp1 = bfp.point[[0,1,2,7,8,19,20,11,12,13,14]]
bfp1[9]  = 1.5 * bfp1[9]  - bfp1[5]
bfp1[10] = 1.5 * bfp1[10] - bfp1[6]
bfp1[5] = -bfp1[5]
bfp1[6] = -bfp1[6]
bfp1[7] = bfp1[5] - 1.5 * bfp1[7]
bfp1[8] = bfp1[6] - 1.5 * bfp1[8]


labels = [
 '$\\mathrm{Re}\\,(\\alpha_1 + \\alpha_4^u)$',
 '$\\mathrm{Re}\\,(\\alpha_2 - \\alpha_4^u)$',
 '$\\mathrm{Im}\\,(\\alpha_2 - \\alpha_4^u)$',
 '$\\mathrm{Re}\\,\\alpha_4^u$',
 '$\\mathrm{Im}\\,\\alpha_4^u$',
 '$\\mathrm{Re}\\,(\\frac{3}{2} \\alpha_{4,EW}^c + \\alpha_4^c)$',
 '$\\mathrm{Im}\\,(\\frac{3}{2} \\alpha_{4,EW}^c + \\alpha_4^c)$',
 '$\\mathrm{Re}\\,\\alpha_4^c$',
 '$\\mathrm{Im}\\,\\alpha_4^c$',
 '$\\mathrm{Re}\\,(\\frac{3}{2} \\alpha_{3,EW}^c - \\alpha_4^c)$',
 '$\\mathrm{Im}\\,(\\frac{3}{2} \\alpha_{3,EW}^c - \\alpha_4^c)$',
]

params = (0,1,2,3,4,7,8)

fsamples = samples[::,params]
fweights = f.weights
labels = np.array(labels)[[params]][0]
bfp1 = bfp1[[params]][0]

size = fsamples.shape[-1]

# Adape the bounds
# bounds =  [(analysis.init_args['priors'][i]['min'], analysis.init_args['priors'][i]['max']) for i in params]
# Main color
color = "C1"
# Show datapoints?
points = False

fig, axes = plt.subplots(size, size, figsize=(3.0 * size, 3.0 * size), dpi=100)

legended_plot = axes[1, 0]

for i in range(size):
    # diagonal
    ax = axes[i, i]
    samples = fsamples[:, i]

    xmin = np.min(samples) #bounds[i][0]
    xmax = np.max(samples) #bounds[i][1]

    kde = gaussian_kde(samples, weights=fweights)
    kde.set_bandwidth(bw_method='silverman')
    kde.set_bandwidth(bw_method=kde.factor * 1.0)

    x = np.linspace(xmin, xmax, 100)
    pdf = kde(x)
    pdf_norm = pdf.sum()

    ax.plot(x, pdf, color=color)

    plevelf = lambda x, pdf, P: pdf[pdf > x * pdf_norm].sum() - P * pdf_norm
    plevel = scipy.optimize.brentq(plevelf, 0., 1., args=(pdf, 68.27 / 100.0 ))
    ax.fill_between(np.ma.masked_array(x, mask=pdf < plevel * pdf_norm),
                    np.ma.masked_array(pdf, mask=pdf < plevel * pdf_norm, fill_value=np.nan),
                    facecolor=color, alpha=0.4)

    ax.set_xlim(left=xmin, right=xmax)
    ax.set_xbound(lower=xmin, upper=xmax)
    ax.set_ylim(bottom=0)
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    ax.yaxis.set_tick_params(left=False)

    # ax.vlines(SM_point[i], ax.get_ylim()[0], ax.get_ylim()[1], color="k")
    ax.vlines(bfp1[i],      ax.get_ylim()[0], ax.get_ylim()[1], ls = 'dotted', color="k")
    # ax.vlines(bfp2[i],      ax.get_ylim()[0], ax.get_ylim()[1], ls = 'dotted', color="k")

    if i == size - 1:
        ax.set_xlabel(labels[i])
    else:
        ax.xaxis.set_major_formatter(plt.NullFormatter())
        ax.xaxis.set_tick_params(bottom=False)


    for j in range(0, size):
        if j > i:
            axes[i, j].set_axis_off()

        if j >= i:
            continue

        ax = axes[i, j]
        xsamples = fsamples[:, j]
        ysamples = fsamples[:, i]
        xmin = np.min(xsamples)#bounds[j][0]
        xmax = np.max(xsamples)#bounds[j][1]
        ymin = np.min(ysamples)#bounds[i][0]
        ymax = np.max(ysamples)#bounds[i][1]
        if points:
            ax.hist2d(xsamples, ysamples, weights=fweights, alpha=1.0, bins=[np.linspace(xmin, xmax, 100), np.linspace(ymin, ymax, 100)], cmap='Greys')

        samples = fsamples[:, (j, i)]
        kde = gaussian_kde(np.transpose(samples), weights=fweights)
        kde.set_bandwidth(bw_method='silverman')
        kde.set_bandwidth(bw_method=kde.factor * 1.0)

        xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])
        pdf = np.reshape(kde(positions).T, xx.shape)
        pdf /= pdf.sum()

        # find the PDF value corresponding to a given cummulative probability
        plevel = lambda x, pdf, P: pdf[pdf > x].sum() - P
        plevels = []
        level_labels = []
        for level in [0, 68, 95, 99]:
            plevels.append(scipy.optimize.brentq(plevel, 0., 1., args=(pdf, level / 100.0)))
            level_labels.append('{}%'.format(level))

        colors = [matplotlib.colors.to_rgba(color, alpha) for alpha in np.linspace(0.20, 0.80, 4)]
        ax.contourf(pdf.transpose(),
                     colors=colors,
                     extent=[xmin, xmax, ymin, ymax],
                     levels=plevels[::-1])

        ax.grid(visible=True, alpha=0.25)

        # ax.plot(SM_point[j], SM_point[i], marker='+', markeredgewidth=3, ls='none', ms=15, alpha=0.5, color="k")
        ax.plot(bfp1[j], bfp1[i],           marker='x', markeredgewidth=3, ls='none', ms=10, alpha=0.5, color="k")

        # Set bounds
        ax.set_xlim(left=xmin, right=xmax)
        ax.set_ylim(bottom=ymin, top=ymax)
        ax.set_xbound(lower=xmin, upper=xmax)
        ax.set_ybound(lower=ymin, upper=ymax)
        # ax.xaxis.set_major_locator(MultipleLocator(0.2))
        # ax.yaxis.set_major_locator(MultipleLocator(0.2))

        if j == 0:
            ax.set_ylabel(labels[i])
        else:
            ax.yaxis.set_major_formatter(plt.NullFormatter())
            ax.yaxis.set_tick_params(left=False)

        if i == size - 1:
            ax.set_xlabel(labels[j])
        else:
            ax.xaxis.set_major_formatter(plt.NullFormatter())
            ax.xaxis.set_tick_params(bottom=False)


# Reticks
#axes[4, 1].set_xticks(np.arange(-0.4, 1.1, 0.4))
#axes[4, 3].set_xticks(np.arange(-0.8, 0.2, 0.4))
#axes[1, 0].set_yticks(np.arange(-0.4, 1.1, 0.4))
#axes[3, 0].set_yticks(np.arange(-0.8, 0.2, 0.4))

# Watermark
xdelta, ydelta = (0.05, 0.04)

ax = axes[size - 1, 0]
version = 'v{version}'.format(version=eos.__version__)
ax.text(xdelta, ydelta, r'\textsf{{\textbf{{EOS {version}}}}}'.format(version=version),
        transform=ax.transAxes,
        color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
        horizontalalignment='left', verticalalignment='bottom', zorder=+5)

# Caption
rec = plt.Rectangle((0,0),1,1, color=color, alpha=0.8)
bfp_point, = plt.plot([0],[-100], marker='x', markeredgewidth=3, ls='none', ms=10, alpha=0.5, color="k")
bfp_line = plt.vlines(1, 0, 0, ls = 'dotted', color="k")


legended_plot.legend(handles=[rec, (bfp_line, bfp_point)], labels = [r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}", "Best fit point"],
                     loc='upper left', handler_map={tuple: HandlerTuple(ndivide=None)})

fig.tight_layout()
plt.savefig("figures/corner_plot_alpha.pdf", bbox_inches='tight', dpi=300)


In [ ]:
def rotate_sample(s):
    theta = np.angle(s[0]-s[7] + 1j*s[8])
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    rot_mat = np.kron(np.eye(12,dtype=int), rot)
    return np.dot(rot_mat, np.concatenate(([s[0]], [0], s[1:])))

In [ ]:
perm = np.array([
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])

In [ ]:
rsamples = np.array([np.dot(np.linalg.inv(perm), rotate_sample(s[:-4])) for s in f.samples])
tr_mean = np.array([ np.average(rsamples[:,i], weights=f.weights) for i in range(rsamples.shape[1])])
tr_std  = np.array([ wstd(rsamples[:,i], weights=f.weights) for i in range(rsamples.shape[1])])
display(tr_mean - 5*tr_std)
display(tr_mean + 5*tr_std)

In [ ]:
Mqcdf_inv = np.array([
              [1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,   -1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,   -1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,   -2.,    0.,  320.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,   -2.,    0.,  320.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0., -320.,    0.,    0.,    0.,    1.,    0.,    0.,    0., -320.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0., -320.,    0.,    0.,    0.,    1.,    0.,    0.,    0., -320.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0., -160.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0., -160.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,  -1.5,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,  -1.5,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,   -1.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,   -1.,    0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  320.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  320.,    0.,    0.,    0.,    1.,    0.,    0.,    0.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,  -1.5,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,  -1.5],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.],
              [0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.]])

Mqcdf = np.linalg.inv(Mqcdf_inv)

In [ ]:
np.dot(Mqcdf_inv, np.dot(np.linalg.inv(perm), rotate_sample(start_point[:-4])))

In [ ]:
trans_samples = np.array([np.dot(Mqcdf_inv, s) for s in rsamples])
trans_cov = np.cov(trans_samples, rowvar=False, aweights=f.weights)
trans_corr = np.array([[trans_cov[i,j] / np.sqrt(trans_cov[i,i] * trans_cov[j,j]) for j in range(len(Mqcdf))] for i in range(len(Mqcdf))])

print(np.max(trans_corr - np.identity(len(Mqcdf))))

tr_mean = np.array([ np.average(trans_samples[:,i], weights=f.weights) for i in range(len(Mqcdf))])
tr_std  = np.array([ wstd(trans_samples[:,i], weights=f.weights) for i in range(len(Mqcdf))])
tr_lo   = tr_mean - np.array([ wquantiles.quantile(trans_samples[:,i], f.weights, 0.5 - 0.34) for i in range(len(Mqcdf))])
tr_hi   = np.array([ wquantiles.quantile(trans_samples[:,i], f.weights, 0.5 + 0.34) for i in range(len(Mqcdf))]) - tr_mean
display(tr_mean - 5*tr_std)
display(tr_mean + 5*tr_std)

In [ ]:
trans_latex = [
    '$\\mathrm{Re}\\,\\alpha_1$',
    '$\\mathrm{Re}\\,\\alpha_2$',
    '$\\mathrm{Im}\\,\\alpha_2$',
    '$\\mathrm{Re}\\,b_1$',
    '$\\mathrm{Im}\\,b_1$',
    '$\\mathrm{Re}\\,b_2$',
    '$\\mathrm{Im}\\,b_2$',
    '$\\mathrm{Re}\\,b_4^u$',
    '$\\mathrm{Im}\\,b_4^u$',
    '$\\mathrm{Re}\\,\\alpha_4^u$',
    '$\\mathrm{Im}\\,\\alpha_4^u$',
    '$\\mathrm{Re}\\,\\alpha_4^c$',
    '$\\mathrm{Im}\\,\\alpha_4^c$',
    '$\\mathrm{Re}\\,\\alpha_{4,EW}^c$',
    '$\\mathrm{Im}\\,\\alpha_{4,EW}^c$',
    '$\\mathrm{Re}\\,\\alpha_{3,EW}^c$',
    '$\\mathrm{Im}\\,\\alpha_{3,EW}^c$',
    '$\\mathrm{Re}\\,b_{3,EW}^c$',
    '$\\mathrm{Im}\\,b_{3,EW}^c$',
    '$\\mathrm{Re}\\,b_{4,EW}^c$',
    '$\\mathrm{Im}\\,b_{4,EW}^c$',
    '$\\mathrm{Re}\\,b_4^c$',
    '$\\mathrm{Im}\\,b_4^c$'
    ]

In [ ]:
for i in range(len(trans_latex)):
    print(f"{trans_latex[i]}: {np.delete(tr_mean,1)[i]:.2f} + {np.delete(tr_hi,1)[i]:.2f} - {np.delete(tr_lo,1)[i]:.2f}")

In [ ]:
samples = np.delete(np.copy(trans_samples), 1, 1)
weights = f.weights
labels = trans_latex
size = samples.shape[-1]

fig, axes = plt.subplots(size, size, figsize=(3.0 * size, 3.0 * size), dpi=100)

for i in range(size):
    # diagonal
    ax = axes[i, i]
    variable_samples = samples[:, i] # For a single variable

    ax.hist(variable_samples, weights=weights, alpha=0.5, bins=100, density=True, stacked=True, color='C1')
    xmin = np.min(variable_samples)
    xmax = np.max(variable_samples)
    ax.set_xlim((xmin, xmax))
    ax.set_xlabel(labels[i])
    ax.set_ylabel(labels[i])
    ax.set_aspect(np.diff((xmin, xmax))[0] / np.diff(ax.get_ylim())[0])

    for j in range(0, size):
        # off-diagonal
        if j < i:
            axes[i, j].set_axis_off()

        if j <= i:
            continue

        ax = axes[i, j]

        # Samples for two single variables as x and y data
        xsamples = samples[:, j]
        ysamples = samples[:, i]

        xmin = np.min(xsamples)
        xmax = np.max(xsamples)
        ymin = np.min(ysamples)
        ymax = np.max(ysamples)
        ax.hist2d(xsamples, ysamples, weights=weights, alpha=1.0, bins=100, cmap='Greys', rasterized=True)
        ax.set_xlim((xmin, xmax))
        ax.set_ylim((ymin, ymax))
        ax.set_aspect(np.diff((xmin, xmax))[0] / np.diff((ymin, ymax))[0])

fig.tight_layout()
fig.show()

#### r theta plot

In [ ]:
def kallen(x, y):
    return np.sqrt((1 - (x + y)**2) * (1 - (x - y)**2))

def acpspiK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    return - 2 * d * np.sin(theta) * np.sin(gamma) \
        / (1 - 2 * d * np.cos(theta) * np.cos(gamma) + d**2)

def acppiK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    eps = 0.053319
    return 2 * eps * d * np.sin(theta) * np.sin(gamma) \
        / (eps**2 + 2 * eps * d * np.cos(theta) * np.cos(gamma) + d**2)

def rpiK(r, theta):
    d = r / 0.389126
    gamma = 1.14761
    eps = 0.053319
    return 0.664 * 0.239 * (1 - 2 * d * np.cos(theta) * np.cos(gamma) + d**2) \
        / (1 + 2 / eps * d * np.cos(theta) * np.cos(gamma) + d**2 / eps**2) \
        / (1 - 0.0635**2) / eps \
        * 1.520 / 5.367 * kallen(0.139/5.367, 0.493/5.367) \
        / (1.517 / 5.279 * kallen(0.139/5.279, 0.493/5.279))


thetabins = np.linspace(2.5, 2.96, 150)
rbins = np.linspace(0.1, 0.3, 150)

theta_mesh, d_mesh = np.meshgrid(thetabins, rbins)
acpspiK_data = acpspiK(d_mesh, theta_mesh)
acppiK_data  = acppiK(d_mesh, theta_mesh)
rpiK_data   = rpiK(d_mesh, theta_mesh)

fig, ax = plt.subplots()

acpspiK_contour = ax.contourf(thetabins, rbins, acpspiK_data, levels = [-0.224 - 0.012,  -0.224 + 0.012], alpha=0.4, cmap="Reds")
acppiK_contour  = ax.contourf(thetabins, rbins, acppiK_data,  levels = [ 0.0831 - 0.0031, 0.0831 + 0.0031],   alpha=0.4, cmap="Greens")
rpiK_contour    = ax.contourf(thetabins, rbins, rpiK_data,    levels = [ 0.074 - 0.008,   0.074 + 0.008],   alpha=0.4, cmap="Blues")

dtheta = np.transpose([np.abs((f.samples[:,19] + 1j * f.samples[:,20]) / f.samples[:,0]), np.angle((f.samples[:,19] + 1j * f.samples[:,20]) / f.samples[:,0])])

kde = gaussian_kde(dtheta[:,(1, 0)].T, weights=f.weights)
kde.set_bandwidth(bw_method='silverman')
kde.set_bandwidth(bw_method=kde.factor * 3)

xx, yy = np.mgrid[thetabins[0]:thetabins[-1]:100j, rbins[0]:rbins[-1]:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
pdf = np.reshape(kde(positions).T, xx.shape)
pdf /= pdf.sum()

# find the PDF value corresponding to a given cummulative probability
plevel = lambda x, pdf, P: pdf[pdf > x].sum() - P
plevels = []
labels = []
for level in [0, 68.3, 95.4, 99.7]:
    plevels.append(brentq(plevel, 0., 1., args=(pdf, level / 100.0)))
    labels.append(f'{level}%')

CS = ax.contour(pdf.transpose(),
            colors="k",
            extent=[thetabins[0], thetabins[-1], rbins[0], rbins[-1]],
            levels=plevels[::-1])


fmt = {}
strs = [r'$3 \sigma$', r'$2 \sigma$', r'$1 \sigma$']
for l, s in zip(CS.levels, strs):
    fmt[l] = s

# Label every other level using strings
plt.clabel(CS, inline=True, fmt=fmt, fontsize=10)

ax.set_xlabel(r'$\theta_p$ [rad]')
ax.set_ylabel(r"$r_p$")

def deg2rad(x): return x * np.pi / 180
def rad2deg(x): return x * 180 / np.pi

secax = ax.secondary_xaxis('top', functions=(rad2deg, deg2rad))
secax.set_xlabel(r'$\theta_p$ [degrees]', labelpad=10)

ax.text(0.04, 0.04, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="left", verticalalignment="bottom", zorder=+5)

handles, labels = plt.gca().get_legend_handles_labels()

proxy = [plt.Rectangle((0, 0), 1, 1, fc = pc.get_facecolor()[0])
         for pc in acpspiK_contour.collections + acppiK_contour.collections + rpiK_contour.collections] \
      + [plt.Line2D((0, 1), (0.5, 0.), color="k")]

ax.set_xbound(2.5, 2.9)
ax.set_ybound(0.1, 0.3)

plt.legend(proxy + handles,
            [r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}(B_s^0\to K^-\pi^+)$", r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}(B^0\to K^+\pi^-)$",
             r"$\frac{f_s}{f_d} \frac{\mathcal{B}(B_s^0\to K^-\pi^+)}{\mathcal{B}(B^0\to K^+\pi^-)}$", r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$} fit"] + labels,
           loc='center right', ncol=1, prop={'size': 13}, bbox_to_anchor=(1.5, 0.5), labelspacing=1)

plt.savefig("./figures/broken_dtheta.pdf")
plt.show()

#### miscellanous ratios with alpha1 > 0

In [ ]:
samples = np.delete(np.copy(trans_samples), 1, 1)
weights = f.weights

In [ ]:
logbins=np.logspace(-5,np.log(2),100)
plt.hist(np.sqrt((samples[::,13]**2 + samples[::,14]**2) / (samples[::, 0]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C6', label=r'$\left|\frac{\alpha_{4,EW}^c}{\alpha_1}\right|$', density=True)
plt.hist(np.sqrt((samples[::,11]**2 + samples[::,12]**2) / (samples[::, 9]**2 + samples[::,10]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C2', label=r'$\left|\frac{\alpha_4^c}{\alpha_4^u}\right|$', density=True)
plt.hist(np.sqrt((samples[::,15]**2 + samples[::,16]**2) / (samples[::, 1]**2 + samples[::, 2]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C0', label=r'$\left|\frac{\alpha_{3,EW}^c}{\alpha_2}\right|$', density=True)
plt.xscale('log')
plt.legend()
plt.show()

In [ ]:
plot_args = {
    'plot': {
        'x': { 'label': r"ratios", 'range': [1e-6, 2], 'scale': 'log' },
        'legend': { 'location': 'upper left' },
        'size': (14,14)
    },
    'contents': [
        {
            'type': 'kde', 'color': 'C1', 'label': r"$\left|\frac{\alpha_{4,EW}^c}{\alpha_1}\right|$",
            'levels': [68, 95], 'contours': ['lines','areas'], 'bandwidth': 1,
            'data': {
                'samples': np.sqrt((samples[::,13]**2 + samples[::,14]**2) / (samples[::, 0]**2)),
                'weights': weights
            }
        },
    ]
}
fig, ax = eos.plot.Plotter(plot_args).plot()

In [ ]:
r_mean = np.array(wquantiles.quantile(samples[::, 3] + 2*samples[::, 7], weights, 0.5))
r_lo   = r_mean - np.array(wquantiles.quantile(samples[::, 3] + 2*samples[::, 7], weights, 0.5 - 0.34))
r_hi   = np.array(wquantiles.quantile(samples[::, 3] + 2*samples[::, 7], weights, 0.5 + 0.34)) - r_mean

print(r_mean, r_lo, r_hi)

r_mean = np.array(wquantiles.quantile(samples[::, 4] + 2*samples[::, 8], weights, 0.5))
r_lo   = r_mean - np.array(wquantiles.quantile(samples[::, 4] + 2*samples[::, 8], weights, 0.5 - 0.34))
r_hi   = np.array(wquantiles.quantile(samples[::, 4] + 2*samples[::, 8], weights, 0.5 + 0.34)) - r_mean

print(r_mean, r_lo, r_hi)

# bins=np.linspace(0,2,100)
plt.hist(samples[::, 3] + 2*samples[::, 7], bins=100, weights=f.weights, alpha=0.5, color='C0', label=r'$\mathrm{Re}\ b_1 + 2 b_4^u$')
plt.hist(samples[::, 4] + 2*samples[::, 8], bins=100, weights=f.weights, alpha=0.5, color='C1', label=r'$\mathrm{Im}\ b_1 + 2 b_4^u$')
plt.legend()
plt.show()

In [ ]:
bins=np.linspace(-1,1,100)
plt.hist(samples[::, 0] + samples[::, 9], bins=bins, weights=f.weights, alpha=0.5, color='C0', label=r'$\mathrm{Re}\ \alpha_1 +  \alpha_4^u$', density=True)
plt.hist(samples[::, 10], bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\mathrm{Im}\ \alpha_1 +  \alpha_4^u$', density=True)

plt.legend()
plt.show()

bins=np.linspace(-1,1,100)
plt.hist(samples[::, 1] - samples[::, 9],  bins=bins, weights=f.weights, alpha=0.5, color='C0', label=r'$\mathrm{Re}\ \alpha_2 -  \alpha_4^u$', density=True)
plt.hist(samples[::, 2] - samples[::, 10], bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\mathrm{Im}\ \alpha_2 -  \alpha_4^u$', density=True)

plt.legend()
plt.show()


In [ ]:
mean = wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 3]**2 + samples[::, 4]**2)), f.weights, 0.5)
lo   = mean - wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 3]**2 + samples[::, 4]**2)), f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 3]**2 + samples[::, 4]**2)), f.weights, 0.5 + 0.34) - mean

print(f"b2/b1 = {mean:.3f} - {lo:.3f} + {hi:.3f}")

In [ ]:
bins=np.linspace(0,2,100)
plt.hist(np.sqrt((samples[::,13]**2 + samples[::,14]**2) / (samples[::,11]**2 + samples[::,12]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{\alpha_{4,EW}^c}{\alpha_4^c}\right|$')
plt.legend()
plt.show()

In [ ]:
plt.hist(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 9]**2 + samples[::,10]**2)), bins=np.linspace(150,500,100), weights=f.weights, alpha=0.5, color='C0', label=r'$\left|\frac{b_2}{\alpha_4^u}\right|$', density=True)


mean = wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 9]**2 + samples[::,10]**2)), f.weights, 0.5)
lo   = mean - wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 9]**2 + samples[::,10]**2)), f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 9]**2 + samples[::,10]**2)), f.weights, 0.5 + 0.34) - mean

print(f"b2/alpha4u = {mean:.3f} + {lo:.3f} - {hi:.3f}")

plt.legend()
plt.show()

In [ ]:
bins=np.linspace(0,2,100)
plt.hist(np.sqrt((samples[::, 5]**2 + samples[::, 6]**2) / (samples[::, 3]**2 + samples[::, 4]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C0', label=r'$\left|\frac{b_2}{b_1}\right|$', density=True)

plt.legend()
plt.show()

bins=np.linspace(0,2,100)
plt.hist(np.sqrt((samples[::, 1]**2 + samples[::, 2]**2) / (samples[::, 0]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{\alpha_2}{\alpha_1}\right|$')
plt.legend()
plt.show()

logbins=np.logspace(-5,np.log(2),100)
plt.hist(np.sqrt((samples[::,19]**2 + samples[::,20]**2) / (samples[::, 3]**2 + samples[::, 4]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C4', label=r'$\left|\frac{b_{4,EW}^c}{b_1}\right|$', density=True)
plt.hist(np.sqrt((samples[::,17]**2 + samples[::,18]**2) / (samples[::, 5]**2 + samples[::, 6]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C6', label=r'$\left|\frac{b_{3,EW}^c}{b_2}\right|$', density=True)
plt.hist(np.sqrt((samples[::,21]**2 + samples[::,22]**2) / (samples[::, 7]**2 + samples[::, 8]**2)), bins=logbins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{b_4^c}{b_4^u}\right|$', density=True)
plt.xscale('log')
plt.legend()
plt.show()


In [ ]:
bins=np.linspace(0,400,100)
plt.hist(np.sqrt((samples[::,21]**2 + samples[::,22]**2) / (samples[::,11]**2 + samples[::,12]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{b_4^c}{\alpha_4^c}\right|$', density=True)

plt.legend()
plt.show()

bins=np.linspace(0,2,100)
plt.hist(np.sqrt((samples[::,11]**2 + samples[::,12]**2) / ((samples[::,0] + samples[::,1])**2 + samples[::,2]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C0', label=r'$\left|\frac{\alpha_4^c}{\alpha_1 + \alpha_2}\right|$', density=True)
plt.hist(np.sqrt((samples[::,15]**2 + samples[::,16]**2) / (samples[::,11]**2 + samples[::,12]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{\alpha_{3,EW}^c}{\alpha_4^c}\right|$', density=True)

plt.legend()
plt.show()

bins=np.linspace(0,2,100)
plt.hist(np.sqrt(((samples[::,11] + 1.5*samples[::,13])**2 + (samples[::,12] + 1.5*samples[::,14])**2) / ((samples[::,0] + samples[::,9])**2 + samples[::,10]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C0', label=r'$\left|\frac{\alpha_4^c}{\alpha_1 + \alpha_4^u}\right|$', density=True)

plt.legend()
plt.show()

In [ ]:
ratios = np.array([
    0.5 * np.log((samples[::,13]**2 + samples[::,14]**2) / (samples[::, 0]**2)),
    0.5 * np.log((samples[::,11]**2 + samples[::,12]**2) / (samples[::, 9]**2 + samples[::,10]**2)),
    0.5 * np.log((samples[::,15]**2 + samples[::,16]**2) / (samples[::, 1]**2 + samples[::, 2]**2)),
    0.5 * np.log((samples[::,19]**2 + samples[::,20]**2) / (samples[::, 3]**2 + samples[::, 4]**2)),
    0.5 * np.log((samples[::,17]**2 + samples[::,18]**2) / (samples[::, 5]**2 + samples[::, 6]**2)),
    0.5 * np.log((samples[::,21]**2 + samples[::,22]**2) / (samples[::, 7]**2 + samples[::, 8]**2))
    ])

r_labels = [
    r'$\ln\left|\frac{\alpha_{4,EW}^c}{\alpha_1}\right|$',
    r'$\ln\left|\frac{\alpha_4^c}{\alpha_4^u}\right|$',
    r'$\ln\left|\frac{\alpha_{3,EW}^c}{\alpha_2}\right|$',
    r'$\ln\left|\frac{b_{4,EW}^c}{b_1}\right|$',
    r'$\ln\left|\frac{b_{3,EW}^c}{b_2}\right|$',
    r'$\ln\left|\frac{b_4^c}{b_4^u}\right|$'
    ]


r_mean = np.array(wquantiles.quantile(ratios, weights, 0.5))
r_lo   = r_mean - np.array(wquantiles.quantile(ratios, weights, 0.5 - 0.34))
r_hi   = np.array(wquantiles.quantile(ratios, weights, 0.5 + 0.34)) - r_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(r_mean)))
ax.errorbar(r_mean, np.arange(len(r_mean)) + 0.1, xerr=[r_lo, r_hi],  color="C2", fmt="v", label=r"data-driven")

ax.yaxis.set_ticks(range(len(r_mean)))
ax.yaxis.set_ticklabels(r_labels)

ax.set_xlabel(r"ratios")
ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(r_mean)))

# ax.set_xlim(0., 2.2)
# ax.hlines(7.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(r_mean) - 0.3)


ax.text(1 - 0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

fig.savefig("./figures/ratio_s.pdf")
fig.show()

#### miscellanous ratios with alpha1 + alpha4u > 0

In [ ]:
samples = np.delete(np.copy(trans_samples), 1, 1)
weights = f.weights

In [ ]:
bins=np.linspace(0,2,100)
plt.hist(np.sqrt((f.samples[::, 1]**2 + f.samples[::, 2]**2) / (f.samples[::, 0]**2)), bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\left|\frac{\alpha_2 - \alpha_4^u}{\alpha_1 + \alpha_4^u}\right|$', density=True)

plt.legend()
plt.show()

In [ ]:
from matplotlib.patches import Wedge

plot_args = {
    'plot': {
        'x': { 'label': r"$\mathrm{Re}\,(\tilde\alpha_1 + \tilde\alpha_2)$", 'range': [ 0.4, 1.5] },
        'y': { 'label': r"$\mathrm{Im}\,(\tilde\alpha_1 + \tilde\alpha_2)$", 'range': [-0.7, 0.2] },
        'legend': { 'location': 'upper left' },
        'size': (14,14)
    },
    'contents': [
        {
            'type': 'errorbar', 'label': r'QCDF', 'elinewidth': 1.0, 'color': 'C0',
            'x': 1.24,   'xerr': [[0.10], [0.16]],
            'y': -0.066, 'yerr': [[0.09], [0.12]],
        },
        {
            'type': 'kde2D', 'color': 'C1', 'label': r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}",
            'levels': [68, 95, 99], 'contours': ['areas'], 'bandwidth': 1.5,
            'data': {
                'samples': np.transpose([f.samples[:, 0], np.zeros(len(f.samples))]) + f.samples[:, (1, 2)],
                'weights': f.weights
            }
        },
    ]
}
fig, ax = eos.plot.Plotter(plot_args).plot()

circle = Wedge((0, 0), 1.03, 0, 360, width=np.sqrt(0.03**2+0.06**2), alpha=0.2, color='g', label=r'$\mathcal{B}(B^+\to\pi^0\pi^+)$', zorder=-1)
ax.add_patch(circle)

bfp_point, = ax.plot([bfp.point[0] + bfp.point[1]],[bfp.point[2]], marker='x', markeredgewidth=3, ls='none', ms=10, alpha=0.5, color="k", label="Best fit point")

handles, labels = plt.gca().get_legend_handles_labels()

proxy = [plt.Rectangle((0, 0), 1, 1, fc = 'C1')]

plt.legend(proxy + handles,
          [r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}"] + labels,
          loc='upper left', ncol=1, prop={'size': 13}, bbox_to_anchor=(0., 1.0), labelspacing=1)

fig.savefig("./figures/alpha12.pdf")

#### Observables

In [ ]:
if debug:
    eos.tasks.predict_observables(af, posterior, "BRs", base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "BRratios",   base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ACPs",      base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "SCPs",      base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ADGs",      base_directory=BASE_DIRECTORY)

    eos.tasks.predict_observables(af, posterior, "BRs_eta",   base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "ACPs_eta",  base_directory=BASE_DIRECTORY)
    eos.tasks.predict_observables(af, posterior, "SCPs_eta",  base_directory=BASE_DIRECTORY)

#### Paper plot

In [ ]:
BRs_light = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-BRs")
BRSU3Fs_light = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-BRs")

BRs_labels = ["$"+eos.Observables()[obs].latex().replace("\\mathcal{B}_\\mathrm{exp}(", "").replace(")", "")+"$" for obs in BRs_light.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['BRs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(BRs_light.lookup_table.keys()), [c['observable'] for c in constraints])

meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

used_meas_mean = np.copy(meas_mean)
used_meas_lo = np.copy(meas_lo)
used_meas_hi = np.copy(meas_hi)

meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")] = 0.766e-6
meas_lo[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")]   = 0.096e-6
meas_hi[list(BRs_light.lookup_table.keys()).index("B_s^0->pi^+pi^-::BR_exp")]   = 0.096e-6

meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")] = 6.19e-6
meas_lo[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")]   = 0.74e-6
meas_hi[list(BRs_light.lookup_table.keys()).index("B_s^0->K^-pi^+::BR_exp")]   = 0.74e-6

meas_mean[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")] = 0.079e-6
meas_lo[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")]   = 0.015e-6
meas_hi[list(BRs_light.lookup_table.keys()).index("B^0->K^+K^-::BR_exp")]   = 0.015e-6

pred_mean = np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRs_light.samples), BRs_light.weights, 0.5 + 0.34)) - pred_mean

SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(BRSU3Fs_light.samples), BRSU3Fs_light.weights, 0.5))
SU3F_pred_lo   = SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(BRSU3Fs_light.samples), BRSU3Fs_light.weights, 0.5 - 0.34))
SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(BRSU3Fs_light.samples), BRSU3Fs_light.weights, 0.5 + 0.34)) - SU3F_pred_mean


meas_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->Kbar^0pi^0::BR_exp")] = pred_mean[list(BRs_light.lookup_table.keys()).index("B_s^0->Kbar^0pi^0::BR_exp")]
mask = meas_hi > 0

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(meas_mean)))
ax.errorbar(meas_mean[mask]/pred_mean[mask], np.arange(len(meas_mean))[mask] - 0.2, xerr=[meas_lo[mask]/pred_mean[mask], meas_hi[mask]/pred_mean[mask]],
            color="grey",  fmt="d")
ax.errorbar(used_meas_mean/pred_mean, np.arange(len(meas_mean)) - 0.2, xerr=[used_meas_lo/pred_mean, used_meas_hi/pred_mean],
            color="k",  fmt="d", label=r"Measurement")
ax.errorbar(SU3F_pred_mean/pred_mean, np.arange(len(meas_mean)) + 0.0, xerr=[SU3F_pred_lo/pred_mean, SU3F_pred_hi/pred_mean],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
ax.errorbar(pred_mean/pred_mean, np.arange(len(meas_mean)) + 0.2, xerr=[pred_lo/pred_mean, pred_hi/pred_mean],
            color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

ax.yaxis.set_ticks(range(len(meas_mean)))
ax.yaxis.set_ticklabels(BRs_labels)

ax.set_xlabel(r"normalized $\mathcal{B}_\mathrm{exp}$")
# ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(meas_mean)))

# ax.set_xlim(0., 2.2)
ax.set_xlim(0.5, 1.5)
ax.hlines(7.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(pred_mean) - 0.3)

for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRs_labels[i]}', 'mean': '{10**6*p:.3f}', 'std low': '{10**6*pred_lo[i]:.3f}', 'std high': '{10**6*pred_hi[i]:.3f}'}}")

ax.text(1 - 0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

fig.savefig("./figures/BRs.pdf")
fig.show()

In [ ]:
ACPs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ACPs")
ACPSU3Fs = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-ACPs")

ACP_labels = ["$"+eos.Observables()[obs].latex().replace("A^\\mathrm{dir}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in ACPs.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['ACPs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(ACPs.lookup_table.keys()), [c['observable'] for c in constraints])

ACP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

ACP_meas_mean[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")] = 0.172
ACP_meas_lo[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")]   = 0.031
ACP_meas_hi[list(ACPs.lookup_table.keys()).index("B_s^0->K^+K^-::A_CP")]   = 0.031

ACP_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5))
ACP_pred_lo   = ACP_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5 - 0.34))
ACP_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPs.samples), ACPs.weights, 0.5 + 0.34)) - ACP_pred_mean

ACP_SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPSU3Fs.samples), ACPSU3Fs.weights, 0.5))
ACP_SU3F_pred_lo   = ACP_SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPSU3Fs.samples), ACPSU3Fs.weights, 0.5 - 0.34))
ACP_SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPSU3Fs.samples), ACPSU3Fs.weights, 0.5 + 0.34)) - ACP_SU3F_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ACP_meas_mean)))
ax.errorbar(ACP_meas_mean, np.arange(len(ACP_meas_mean)) - 0.2, xerr=[ACP_meas_lo, ACP_meas_hi],
            color="k",  fmt="d", label=r"Measurement")
ax.errorbar(ACP_SU3F_pred_mean, np.arange(len(ACP_meas_mean)) + 0.0, xerr=[ACP_SU3F_pred_lo, ACP_SU3F_pred_hi],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
ax.errorbar(ACP_pred_mean, np.arange(len(ACP_meas_mean)) + 0.2, xerr=[ACP_pred_lo, ACP_pred_hi],
            color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

ax.yaxis.set_ticks(range(len(ACP_meas_mean)))
ax.yaxis.set_ticklabels(ACP_labels)

ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}$")
# ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(ACP_meas_mean)))

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(7.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(ACP_pred_mean) - 0.3)

ax.text(1 - 0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

for i, p in enumerate(ACP_pred_mean):
    print(f"      - {{ 'decay': '{ACP_labels[i]}', 'mean': '{p*100:.2f}', 'std low': '{ACP_pred_lo[i]*100:.2f}', 'std high': '{ACP_pred_hi[i]*100:.2f}'}}")

fig.savefig("./figures/ACPs.pdf")
fig.show()

In [ ]:
SCPs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-SCPs")
SCPSU3Fs = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-SCPs")

SCP_labels = ["$"+eos.Observables()[obs].latex().replace("A^\\mathrm{mix}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in SCPs.lookup_table.keys()]

exp_const   = [const.constraint for const in af.likelihoods['SCPs'].constraints]
constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(SCPs.lookup_table.keys()), [c['observable'] for c in constraints])

SCP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
SCP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
SCP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

SCP_meas_mean[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")] = -0.139
SCP_meas_lo[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.032
SCP_meas_hi[list(SCPs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.032

SCP_pred_mean = np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5))
SCP_pred_lo   = SCP_pred_mean - np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5 - 0.34))
SCP_pred_hi   = np.array(wquantiles.quantile(np.transpose(SCPs.samples), SCPs.weights, 0.5 + 0.34)) - SCP_pred_mean

SCP_SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(SCPSU3Fs.samples), SCPSU3Fs.weights, 0.5))
SCP_SU3F_pred_lo   = SCP_SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(SCPSU3Fs.samples), SCPSU3Fs.weights, 0.5 - 0.34))
SCP_SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(SCPSU3Fs.samples), SCPSU3Fs.weights, 0.5 + 0.34)) - SCP_SU3F_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(SCP_labels)))
ax.errorbar(SCP_meas_mean, np.arange(len(SCP_meas_mean)) - 0.2, xerr=[SCP_meas_lo, SCP_meas_hi],
            color="k",  fmt="d", label=r"Measurement")
ax.errorbar(SCP_SU3F_pred_mean, np.arange(len(SCP_meas_mean)) + 0.0, xerr=[SCP_SU3F_pred_lo, SCP_SU3F_pred_hi],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
ax.errorbar(SCP_pred_mean, np.arange(len(SCP_meas_mean)) + 0.2, xerr=[SCP_pred_lo, SCP_pred_hi],
            color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

ax.yaxis.set_ticks(range(len(SCP_labels)))
ax.yaxis.set_ticklabels(SCP_labels)

ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{mix}$")
# ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(SCP_labels)))

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(4.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(SCP_pred_mean) - 0.3)

ax.text(0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="left", verticalalignment="top", zorder=+5)

for i, p in enumerate(SCP_pred_mean):
    print(f"      - {{ 'decay': '{SCP_labels[i]}', 'median': '{p*100:.2f}', 'std low': '{SCP_pred_lo[i]*100:.2f}', 'std high': '{SCP_pred_hi[i]*100:.2f}'}}")

fig.savefig("./figures/SCPs.pdf")
fig.show()

In [ ]:
ADGs = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ADGs")
# ADGSU3Fs = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-ADGs")

ADG_labels = ["$"+eos.Observables()[obs].latex().replace("A^{\\Delta\\Gamma}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in ADGs.lookup_table.keys()]

# exp_const   = [const.constraint for const in af.likelihoods['ADGs'].constraints]
# constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

# tab = match_table(list(ADGs.lookup_table.keys()), [c['observable'] for c in constraints])

# ADG_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
# ADG_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
# ADG_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

# ADG_meas_mean[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")] = -0.083
# ADG_meas_lo[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.010
# ADG_meas_hi[list(ADGs.lookup_table.keys()).index("B_s^0->K^+K^-::S_CP")]   =  0.010

ADG_pred_mean = np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5))
ADG_pred_lo   = ADG_pred_mean - np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5 - 0.34))
ADG_pred_hi   = np.array(wquantiles.quantile(np.transpose(ADGs.samples), ADGs.weights, 0.5 + 0.34)) - ADG_pred_mean

# ADG_SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(ADGSU3Fs.samples), ADGSU3Fs.weights, 0.5))
# ADG_SU3F_pred_lo   = ADG_SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(ADGSU3Fs.samples), ADGSU3Fs.weights, 0.5 - 0.34))
# ADG_SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(ADGSU3Fs.samples), ADGSU3Fs.weights, 0.5 + 0.34)) - ADG_SU3F_pred_mean

# fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ADG_labels)))
# ax.errorbar(ADG_meas_mean, np.arange(len(ADG_meas_mean)) - 0.2, xerr=[ADG_meas_lo, ADG_meas_hi],
#             color="k",  fmt="d", label=r"Measurement")
# ax.errorbar(ADG_SU3F_pred_mean, np.arange(len(ADG_meas_mean)) + 0.0, xerr=[ADG_SU3F_pred_lo, ADG_SU3F_pred_hi],
#             color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
# ax.errorbar(ADG_pred_mean, np.arange(len(ADG_meas_mean)) + 0.2, xerr=[ADG_pred_lo, ADG_pred_hi],
#             color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

# ax.yaxis.set_ticks(range(len(ADG_labels)))
# ax.yaxis.set_ticklabels(ADG_labels)

# ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{mix}$")
# # ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(ADG_labels)))

# ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
# ax.hlines(4.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
# ax.set_ylim(-0.5, len(ADG_pred_mean) - 0.3)

# ax.text(0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
#                     transform=ax.transAxes,
#                     color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
#                     horizontalalignment="left", verticalalignment="top", zorder=+5)

for i, p in enumerate(ADG_pred_mean):
    print(f"      - {{ 'decay': '{ADG_labels[i]}', 'median': '{p*100:.2f}', 'std low': '{ADG_pred_lo[i]*100:.2f}', 'std high': '{ADG_pred_hi[i]*100:.2f}'}}")

# fig.savefig("./figures/ADGs.pdf")
# fig.show()

In [ ]:
BRratios = eos.Prediction(BASE_DIRECTORY+posterior+"/pred-BRratios")
BRSU3Fratios = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-BRratios")

BRratio_labels = ["$"+eos.Observables()[obs].latex()+"$" for obs in BRratios.lookup_table.keys()]

exp_const   = [const.name for const in af._likelihoods['BRratios'].manual_constraints]
constraints = [const.info for const in af._likelihoods['BRratios'].manual_constraints]

BRratios_mean = np.array([float(c["mean"]) for c in constraints])
BRratios_lo   = np.array([np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints])
BRratios_hi   = np.array([np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints])

pred_mean = np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRratios.samples), BRratios.weights, 0.5 + 0.34)) - pred_mean

SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(BRSU3Fratios.samples), BRSU3Fratios.weights, 0.5))
SU3F_pred_lo   = SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(BRSU3Fratios.samples), BRSU3Fratios.weights, 0.5 - 0.34))
SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(BRSU3Fratios.samples), BRSU3Fratios.weights, 0.5 + 0.34)) - SU3F_pred_mean


fig, ax = plt.subplots(ncols=1, figsize=(6, 1.25 * len(BRratio_labels)))
ax.errorbar(BRratios_mean/pred_mean, np.arange(len(BRratio_labels)) - 0.2, xerr=[BRratios_lo/pred_mean, BRratios_hi/pred_mean],
            color="k",  fmt="d", label=r"Measurement")
ax.errorbar(SU3F_pred_mean/pred_mean, np.arange(len(BRratio_labels)) + 0.0, xerr=[SU3F_pred_lo/pred_mean, SU3F_pred_hi/pred_mean],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
ax.errorbar(pred_mean/pred_mean, np.arange(len(BRratio_labels)) + 0.2, xerr=[pred_lo/pred_mean, pred_hi/pred_mean],
            color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")

ax.yaxis.set_ticks(range(len(BRratio_labels)))
ax.yaxis.set_ticklabels(BRratio_labels)

ax.set_xlabel(r"normalized ratios")
ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 0.95 + 1. / len(BRratio_labels)))

ax.set_ylim(-0.5, len(pred_mean) - 0.5)

ax.text(1 - 0.04, 1 - 0.03, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRratio_labels[i]}', 'mean': '{p:.5f}', 'std low': '{pred_lo[i]:.5f}', 'std high': '{pred_hi[i]:.5f}'}}")

fig.savefig("./figures/BRratios.pdf")
fig.show()

In [ ]:
BRs_eta = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-BRs_eta")
BRSU3Fs_eta = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-BRs_eta")

BRs_eta_labels = ["$"+eos.Observables()[obs].latex().replace("\\mathcal{B}_\\mathrm{exp}(", "").replace(")", "")+"$" for obs in BRs_eta.lookup_table.keys()]

exp_const   = ['B^+->etapi^+::BR_exp@PDG:2024A',
 'B^0->etapi^0::BR_exp@Belle:2015B',
 'B^+->etaK^+::BR_exp@PDG:2024A',
 'B^0->etaeta::BR_exp@BaBar:2009B',
 'B_s^0->etaeta::BR_exp@Belle:2021A',
 'B^0->etaK^0::BR_exp@PDG:2024A'
 ]

constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(BRs_eta.lookup_table.keys()), [c['observable'] for c in constraints])

meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

pred_mean = np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5))
pred_lo   = pred_mean - np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5 - 0.34))
pred_hi   = np.array(wquantiles.quantile(np.transpose(BRs_eta.samples), BRs_eta.weights, 0.5 + 0.34)) - pred_mean

SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(BRSU3Fs_eta.samples), BRSU3Fs_eta.weights, 0.5))
SU3F_pred_lo   = SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(BRSU3Fs_eta.samples), BRSU3Fs_eta.weights, 0.5 - 0.34))
SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(BRSU3Fs_eta.samples), BRSU3Fs_eta.weights, 0.5 + 0.34)) - SU3F_pred_mean

meas_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etaK^0::BR_exp")] = pred_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etaK^0::BR_exp")]
meas_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etapi^0::BR_exp")] = pred_mean[list(BRs_eta.lookup_table.keys()).index("B_s^0->etapi^0::BR_exp")]
mask = meas_hi > 0

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(meas_mean)))
ax.errorbar(meas_mean[mask]/SU3F_pred_mean[mask], np.arange(len(SU3F_pred_mean))[mask] - 0.1, xerr=[meas_lo[mask]/SU3F_pred_mean[mask], meas_hi[mask]/SU3F_pred_mean[mask]],
            color="grey",  fmt="d", label="Measurement")
ax.errorbar(SU3F_pred_mean/SU3F_pred_mean, np.arange(len(SU3F_pred_mean)) + 0.1, xerr=[SU3F_pred_lo/SU3F_pred_mean, SU3F_pred_hi/SU3F_pred_mean],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")

ax.yaxis.set_ticks(range(len(meas_mean)))
ax.yaxis.set_ticklabels(BRs_eta_labels)

ax.set_xlabel(r"normalized $\mathcal{B}_\mathrm{exp}$")
ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(BRs_eta_labels)))

ax.set_xlim(ax.get_xlim()[0], 12)
ax.hlines(3.5, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(pred_mean) - 0.3)

for i, p in enumerate(pred_mean):
    print(f"      - {{ 'decay': '{BRs_eta_labels[i]}', 'mean': '{p}', 'std low': '{pred_lo[i]}', 'std high': '{pred_hi[i]}'}}")

ax.text(1 - 0.04, 1 - 0.04, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

fig.savefig("./figures/BRs_eta.pdf")
fig.show()

In [ ]:
ACPs_eta = eos.Prediction(BASE_DIRECTORY+"/"+posterior+"/pred-ACPs_eta")
ACPSU3Fs_eta = eos.Prediction(BASE_DIRECTORY+"Topological_no_eta_CKM/pred-ACPs_eta")

ACP_eta_labels = ["$"+eos.Observables()[obs].latex().replace("A^\\mathrm{dir}_\\mathrm{CP}(", "").replace(")", "")+"$" for obs in ACPs_eta.lookup_table.keys()]

exp_const   = ['B^+->etaK^+::A_CP@PDG:2024A',
 'B^+->etapi^+::A_CP@PDG:2024A'
]

constraints = [yaml.safe_load(eos.Constraints()[obs].serialize()) for obs in exp_const]

tab = match_table(list(ACPs_eta.lookup_table.keys()), [c['observable'] for c in constraints])

ACP_meas_mean = np.array([[float(c["mean"]) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_lo   = np.array([[np.sqrt(float(c["sigma-stat"]["lo"])**2 + float(c["sigma-sys"]["lo"])**2) for c in constraints][i] if i != None else np.nan for i in tab])
ACP_meas_hi   = np.array([[np.sqrt(float(c["sigma-stat"]["hi"])**2 + float(c["sigma-sys"]["hi"])**2) for c in constraints][i] if i != None else np.nan for i in tab])

ACP_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5))
ACP_pred_lo   = ACP_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5 - 0.34))
ACP_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPs_eta.samples), ACPs_eta.weights, 0.5 + 0.34)) - ACP_pred_mean

ACP_SU3F_pred_mean = np.array(wquantiles.quantile(np.transpose(ACPSU3Fs_eta.samples), ACPSU3Fs_eta.weights, 0.5))
ACP_SU3F_pred_lo   = ACP_SU3F_pred_mean - np.array(wquantiles.quantile(np.transpose(ACPSU3Fs_eta.samples), ACPSU3Fs_eta.weights, 0.5 - 0.34))
ACP_SU3F_pred_hi   = np.array(wquantiles.quantile(np.transpose(ACPSU3Fs_eta.samples), ACPSU3Fs_eta.weights, 0.5 + 0.34)) - ACP_SU3F_pred_mean

fig, ax = plt.subplots(ncols=1, figsize=(6, 0.75 * len(ACP_meas_mean)))
ax.errorbar(ACP_SU3F_pred_mean, np.arange(len(ACP_meas_mean)) + 0.1, xerr=[ACP_SU3F_pred_lo, ACP_SU3F_pred_hi],
            color="C0", fmt="o", label=r"$\mathrm{SU}(3)_\mathrm{F}$")
# ax.errorbar(ACP_pred_mean, np.arange(len(ACP_meas_mean)) - 0.1, xerr=[ACP_pred_lo, ACP_pred_hi],
#             color="C1", fmt="v", label=r"Fact.-\sout{$\mathrm{SU}(3)_\mathrm{F}$}")
ax.errorbar(ACP_meas_mean, np.arange(len(ACP_meas_mean)) - 0.1, xerr=[ACP_meas_lo, ACP_meas_hi],
            color="grey",  fmt="d", label=r"Measurement")

ax.yaxis.set_ticks(range(len(ACP_meas_mean)))
ax.yaxis.set_ticklabels(ACP_eta_labels)

ax.set_xlabel(r"$\mathcal{A}_\mathrm{CP}^\mathrm{dir}$")
# ax.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.0 + 1. / len(ACP_meas_mean)))

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])
ax.hlines(3.6, ax.get_xlim()[0], ax.get_xlim()[1], lw=1, color="grey", ls="--")
ax.set_ylim(-0.5, len(ACP_pred_mean) - 0.4)

ax.text(1 - 0.04, 1 - 0.02, fr'\textsf{{\textbf{{EOS v{eos.__version__}}}}}',
                    transform=ax.transAxes,
                    color='OrangeRed', alpha=0.5, bbox=dict(facecolor='white', alpha=0.5, lw=0),
                    horizontalalignment="right", verticalalignment="top", zorder=+5)

for i, p in enumerate(ACP_pred_mean):
    print(f"      - {{ 'decay': '{ACP_eta_labels[i]}', 'mean': '{p*100}', 'std low': '{ACP_pred_lo[i]*100}', 'std high': '{ACP_pred_hi[i]*100}'}}")

fig.savefig("./figures/ACPs_eta.pdf")
fig.show()

#### SR

In [ ]:
tauB0 = analysis.parameters["life_time::B_d"].evaluate()
tauBp = analysis.parameters["life_time::B_s"].evaluate()

DeltaSR = - 2.0 * ACPs.samples[:,7] * BRs_light.samples[:,7] / BRs_light.samples[:,5] * tauB0 / tauBp \
    - 2.0 * ACPs.samples[:,4] * BRs_light.samples[:,4] / BRs_light.samples[:,5] \
    + ACPs.samples[:,5] \
    + ACPs.samples[:,6] * BRs_light.samples[:,6] / BRs_light.samples[:,5] * tauB0 / tauBp

mean = wquantiles.quantile(DeltaSR, f.weights, 0.5)
lo   = mean - wquantiles.quantile(DeltaSR, f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(DeltaSR, f.weights, 0.5 + 0.34) - mean

print(f"Delta SR = {mean:.3f} + {lo:.3f} - {hi:.3f}")

bins=np.linspace(-.2,.4,100)
plt.hist(DeltaSR, bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\Delta_\mathrm{SR}$', density=True)

plt.legend()
plt.show()

In [ ]:
deltaSR = ACPs.samples[:,7] - ACPs.samples[:,5]

mean = wquantiles.quantile(deltaSR, f.weights, 0.5)
lo   = mean - wquantiles.quantile(deltaSR, f.weights, 0.5 - 0.34)
hi   = wquantiles.quantile(deltaSR, f.weights, 0.5 + 0.34) - mean

print(f"Delta SR = {mean:.3f} - {lo:.3f} + {hi:.3f}")

bins=np.linspace(-.2,.4,100)
plt.hist(deltaSR, bins=bins, weights=f.weights, alpha=0.5, color='C1', label=r'$\Delta_\mathrm{SR}$', density=True)

plt.legend()
plt.show()